## Loading the libraries

In [144]:
%classpath add mvn com.salesforce.transmogrifai transmogrifai-core_2.11 0.6.0

In [131]:
%classpath add mvn org.apache.spark spark-mllib_2.11 2.3.2

## Import the classes

In [132]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.sql.functions.udf

import com.salesforce.op._
import com.salesforce.op.features._
import com.salesforce.op.features.types._
import com.salesforce.op.stages.impl.classification._
import com.salesforce.op.evaluators.Evaluators

import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.sql.functions.udf
import com.salesforce.op._
import com.salesforce.op.features._
import com.salesforce.op.features.types._
import com.salesforce.op.stages.impl.classification._
import com.salesforce.op.evaluators.Evaluators


In [133]:
import com.salesforce.op.OpWorkflow
import com.salesforce.op.evaluators.Evaluators
import com.salesforce.op.readers.DataReaders

import com.salesforce.op.OpWorkflow
import com.salesforce.op.evaluators.Evaluators
import com.salesforce.op.readers.DataReaders


## Instantiate Spark

In [134]:
val conf = new SparkConf().setMaster("local[*]").setAppName("Airbnb analysis")
implicit val spark = SparkSession.builder.config(conf).getOrCreate()

org.apache.spark.sql.SparkSession@41a992e4

## Schema and Feature Creation

In [135]:
//room_type	neighbourhood	number_of_reviews	price
case class SimpleRegression (
  roomType: String,
  neighbourhood: String,
  numberOfReviews: Integer,
  price: Double
)

defined class SimpleRegression


In [136]:
//val population = FeatureBuilder.RealNN[SimpleRegression].extract(_.population.toRealNN).asPredictor
val roomType = FeatureBuilder.Text[SimpleRegression].extract(_.roomType.toText).asPredictor
val neighbourhood = FeatureBuilder.Text[SimpleRegression].extract(_.neighbourhood.toText).asPredictor
val numberOfReviews = FeatureBuilder.Integral[SimpleRegression].extract(_.numberOfReviews.toIntegral).asPredictor
//val numberOfReviews = FeatureBuilder.RealNN[SimpleRegression].extract(
//     v => Option(v.numberOfReviews).toRealNN(throw new Exception("no of reviews cannot be null"))
//).asPredictor
val price = FeatureBuilder.RealNN[SimpleRegression].extract(_.price.toRealNN).asResponse

Feature(name = price, uid = RealNN_0000000000f1, isResponse = true, originStage = FeatureGeneratorStage_0000000000f1, parents = [], distributions = [])

## Load the data

In [137]:
import spark.implicits._

val trainFilePath = "./data/listing_three_features.csv"
val trainDataReader = DataReaders.Simple.csvCase[SimpleRegression](
    path = Option(trainFilePath)
)

org.apache.spark.sql.SparkSession$implicits$@229487f0

In [138]:
// check that path exists
scala.reflect.io.File(trainFilePath).exists

true

In [139]:
import com.salesforce.op.stages.impl.tuning.{DataCutter, DataSplitter}
val features = Seq(roomType, neighbourhood,numberOfReviews).transmogrify()
val randomSeed = 42L
val splitter = DataSplitter(seed = randomSeed)

DataSplitter_0000000000f5

## Model Selector
The ModelSelector is an Estimator that uses data to find the best model. 

In [140]:
//import com.salesforce.op.stages.impl.regression.RegressionModelsToTry.{OpGBTRegressor, OpRandomForestRegressor,OpLinearRegression}
import com.salesforce.op.stages.impl.regression.RegressionModelsToTry.{OpLinearRegression}
import com.salesforce.op.stages.impl.regression.RegressionModelSelector

val cutter = DataCutter(reserveTestFraction = 0.2, seed = randomSeed)

val prediction = RegressionModelSelector
      .withCrossValidation(
        dataSplitter = Some(splitter), seed = randomSeed,
        //modelTypesToUse = Seq(OpGBTRegressor, OpRandomForestRegressor)
        //modelTypesToUse = Seq(OpLinearRegression, 
        //                      OpRandomForestRegressor)
        modelTypesToUse = Seq(OpLinearRegression)
        
 
).setInput(price,features).getOutput()

Feature(name = neighbourhood-numberOfReviews-price-roomType_4-stagesApplied_Prediction_0000000000ff, uid = Prediction_0000000000ff, isResponse = true, originStage = ModelSelector_0000000000ff, parents = [RealNN_0000000000f1,OPVector_0000000000f4], distributions = [])

## Evaluators and Workflow
Factory that performs the evaluation of metrics for regression. The metrics returned are rmse, mse, r2 and mae.
* Mean Squared Error (MSE)	
* Root Mean Squared Error (RMSE)	
* Mean Absolute Error (MAE)	
* Coefficient of Determination 

OpWorkflows create and transform the raw data needed to compute Features fed into them. In addition they optimize the application of Stages needed to create the final Features ensuring optimal computations within the full pipeline DAG. OpWorkflows can be fit to a given dataset using the .train() method. This produces an OpWorkflowModel which can then be saved to disk and applied to another dataset.

In [141]:
val evaluator = Evaluators.Regression().setLabelCol(price).
      setPredictionCol(prediction)

OpRegressionEvaluator_000000000100

In [142]:
val workflow = new OpWorkflow().setResultFeatures(prediction, price).setReader(trainDataReader)
val workflowModel = workflow.train()

com.salesforce.op.OpWorkflowModel@1d4ac211

## Score and evaluate

In [143]:
val dfScoreAndEvaluate = workflowModel.scoreAndEvaluate(evaluator)
val dfScore = dfScoreAndEvaluate._1//.withColumnRenamed("population-profit_3-stagesApplied_Prediction_000000000011","predicted_profit")
val dfEvaluate = dfScoreAndEvaluate._2
println("Evaluate:\n" + dfEvaluate.toString())

dfScore.show(false)

Evaluate:
{
  "RootMeanSquaredError" : 65.28712197274389,
  "MeanSquaredError" : 4262.408295483939,
  "R2" : 0.4262188601395154,
  "MeanAbsoluteError" : 49.10125039144317
}
+--------------------+-----+------------------------------------------------------------------------------------+
|key                 |price|neighbourhood-numberOfReviews-price-roomType_4-stagesApplied_Prediction_0000000000ff|
+--------------------+-----+------------------------------------------------------------------------------------+
|1983412920050712762 |83.0 |[prediction -> 64.17323999628853]                                                   |
|8857413378039979245 |81.0 |[prediction -> 87.35390817344154]                                                   |
|6553189074429511325 |69.0 |[prediction -> 61.9150261033396]                                                    |
|2661168433446230458 |206.0|[prediction -> 75.16458872759063]                                                   |
|-6564297660568305272|94.0 |[

null